In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

import numpy as np
#import jplus



Using TensorFlow backend.
/Users/aaorsi/anaconda/lib/python2.7/site-packages/cryptography/hazmat/primitives/constant_time.py:26: CryptographyDeprecationWarning: Support for your Python version is deprecated. The next version of cryptography will remove support. Please upgrade to a 2.7.x release that supports hmac.compare_digest as soon as possible.
  utils.DeprecatedIn23,


In [2]:
#load databases

#DataDir = './data/'
DataDir = '.'
import pickle
dset = pickle.load(open('%s/training_full.data'%DataDir))
print dset.keys()


featnames = ['dm_j0660','J0660','uSDSS','gSDSS','rSDSS','iSDSS','zSDSS']

ntrain = len(dset['obj'])
feat_arr = []

for x in range(ntrain):
    fx = []
    for y in featnames:
        fx.append(dset[y][x,0])
    fx.append(dset['uSDSS'][x,0]-dset['gSDSS'][x,0])
    fx.append(dset['gSDSS'][x,0]-dset['rSDSS'][x,0])
    fx.append(dset['rSDSS'][x,0]-dset['iSDSS'][x,0])
    fx.append(dset['uSDSS'][x,0]-dset['gSDSS'][x,0])
    fx.append(dset['iSDSS'][x,0]-dset['zSDSS'][x,0])
    fx.append(dset['uSDSS'][x,0]-dset['zSDSS'][x,0])
    fx.append(dset['rSDSS'][x,0]-dset['zSDSS'][x,0])
    
    feat_arr.append(fx)

class_arr = dset['class']



['rSDSS', 'iSDSS', 'obj', 'gSDSS', 'zSDSS', 'uSDSS', 'dm_j0660', 'J0660', 'class']


In [8]:
print feat_arr[0]

[1.1525213546782176, 21.1055641, 22.90729, 22.27916, 22.05294, 21.66249, 21.61529, 0.6281299999999987, 0.22622000000000142, 0.3904500000000013, 0.6281299999999987, 0.04719999999999658, 1.291999999999998, 0.4376499999999979]


In [29]:
#Example sets

def class_to_int(istr):
    if istr == 'Halpha':
        return 0
    elif istr == 'OIII+Hbeta':
        return 1
    if istr == 'OII':
        return 2
    elif istr == 'contaminant':
        return 3
    else:
        print '%s not recognised'%istr
        return -99


y_train_int = [class_to_int(x) for x in class_arr]
y_train = keras.utils.to_categorical(y_train_int, num_classes=len(np.unique(y_train_int)))
x_train = feat_arr
nfeat = len(feat_arr[0])
print nfeat
print y_train_int



14
[1, 2, 1, 2, 2, 2, 0, 1, 2, 2, 1, 1, 2, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 0, 1, 0, 2, 1, 2, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 2, 0, 1, 1, 0, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

Scale data using scikit-learn StandardScaler

In [25]:
from sklearn.preprocessing import StandardScaler 
Scaledata = True

if Scaledata:
    print 'scaling data...',
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    #x_test  = scaler.transform(x_test)
    print 'done'


scaling data... done


In [28]:
# Keras MLP

from keras import metrics

model = Sequential()
model.add(Dense(19, activation='relu', input_dim=nfeat))
model.add(Dense(19, activation='relu'))
model.add(Dense(4, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=[metrics.categorical_accuracy])
print 'doing stuff'

print np.shape(x_train)
print np.shape(y_train)

model.summary()



doing stuff
(4031, 14)
(4031, 4)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 19)                285       
_________________________________________________________________
dense_8 (Dense)              (None, 19)                380       
_________________________________________________________________
dense_9 (Dense)              (None, 4)                 80        
Total params: 745
Trainable params: 745
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.fit(x_train, y_train,
          epochs=20,
          batch_size=500,
          shuffle=True,
          verbose=1
          )



Epoch 1/20
4031/4031 [==============================] - 0s 18us/step - loss: 0.1244 - categorical_accuracy: 0.9715
Epoch 2/20
4031/4031 [==============================] - 0s 20us/step - loss: 0.1240 - categorical_accuracy: 0.9715
Epoch 3/20
4031/4031 [==============================] - 0s 16us/step - loss: 0.1235 - categorical_accuracy: 0.9715
Epoch 4/20
4031/4031 [==============================] - 0s 17us/step - loss: 0.1230 - categorical_accuracy: 0.9715
Epoch 5/20
4031/4031 [==============================] - 0s 20us/step - loss: 0.1226 - categorical_accuracy: 0.9715
Epoch 6/20
4031/4031 [==============================] - 0s 17us/step - loss: 0.1222 - categorical_accuracy: 0.9715
Epoch 7/20
4031/4031 [==============================] - 0s 31us/step - loss: 0.1217 - categorical_accuracy: 0.9715
Epoch 8/20
4031/4031 [==============================] - 0s 30us/step - loss: 0.1213 - categorical_accuracy: 0.9715
Epoch 9/20
4031/4031 [==============================] - 0s 23us/step - loss: 0.1

In [18]:
pred = model.predict(x_train,verbose=1)
pred_class = [np.argmax(x) for x in pred]
#print x_train[0:10]


4031/4031 [==============================] - 0s 56us/step


In [20]:
from sklearn.metrics import f1_score
score = f1_score(y_train_int, pred_class,average=None)
print score
print np.unique(pred_class)

[0.         0.         0.         0.98552913]
[3]
